Câu 6. Kiểm định ANOVA (2 điểm)
Tập tin ‘Samples.txt’ lưu trữ 4 mẫu dữ liệu được lấy từ các quần thể đều có phân phối chuẩn.
6.1) Với  = 0.05, hãy kiểm định giả thuyết H0: “Các quần thể có cùng phương sai.”
6.2) Với  = 0.05, hãy cho kết luận về giả thuyết H0: “Các quần thể có cùng giá trị trung bình.”
bằng 2 phương pháp: a) Tính toán truyền thống, VÀ b) Dùng các hàm thống kê có sẵn.
6.3) Nếu bác bỏ giả thuyết H0 trong câu 6.2), hãy cho biết những quần thể nào có sự khác biệt về
giá trị trung bình.

In [1]:
import matplotlib.pyplot as plt
import pandas            as pd
import scipy.stats       as stats
import seaborn           as sns
import statsmodels.api   as sm
import numpy       as np
from statsmodels.formula.api     import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd


In [2]:
alpha      = .05
confidence = (1 - alpha)
df=pd.read_csv('Samples.txt',sep='\t')
df

,A,B,C,D
0,93,85.0,100.0,96.0
1,120,45.0,75.0,58.0
2,65,80.0,65.0,95.0
3,105,28.0,40.0,90.0
4,115,75.0,73.0,65.0
5,82,70.0,65.0,80.0
6,99,65.0,50.0,85.0
7,87,55.0,30.0,95.0
8,100,50.0,45.0,82.0
9,90,40.0,50.0,NaN


In [33]:
#6.1. Kiểm định Levene: A,B,C,D có cùng phương sai
print('-------------------------------------------------')
print('* Kiểm định LEVENE:                              ')
print('    H0: VAR(A) = VAR(B) = VAR(C) = VAR(D)  ')
print('    Ha: Các phương sai KHÔNG BẰNG NHAU           ')
print('-------------------------------------------------')
a=df.A.dropna()
b=df.B.dropna()
c=df.C.dropna()
d=df.D.dropna()
levene, pvalue = stats.levene(a, b, c, d, center='mean')
print('* Levene-statistic = %.4f, p-value = %.4f' % (levene, pvalue))
# p-value > alpha => không bác bỏ H0: VAR(A) = VAR(B) = VAR(C) = VAR(D)
if (pvalue >= alpha):
    print('(pvalue >  alpha) ==> ACCEPT the H0 that the variances are equal.')
else:
    print('(pvalue <= alpha) ==> REJECT the H0 that the variances are equal.')

-------------------------------------------------
* Kiểm định LEVENE:                              
    H0: VAR(A) = VAR(B) = VAR(C) = VAR(D)  
    Ha: Các phương sai KHÔNG BẰNG NHAU           
-------------------------------------------------
* Levene-statistic = 0.9568, p-value = 0.4220
(pvalue >  alpha) ==> ACCEPT the H0 that the variances are equal.


In [34]:
#6.2. Áp dụng ANOVA

k      = len(df.columns)
groups = list(df.columns)

samples = []
for j in range(k):
    sample = [x for x in df[groups[j]] if pd.notnull(x)]
    samples.append(sample)

##------------------------------------------------------------------------------
print('----------- Cách 1: Tính toán "truyền thống" theo các công thức ---------')
##------------------------------------------------------------------------------    

sizes = np.zeros(k)
for j in range(k): 
    sizes[j] = np.size(samples[j])

means = np.zeros(k)
for j in range(k):
    means[j] = np.mean(samples[j])

meanT = np.mean(means)

SSB = 0
for j in range(k):
    SSB += sizes[j] * np.power((means[j] - meanT), 2)
dfB = (k - 1)
print('SSB           : %.4f' %SSB)
print('dfB           : %d' %dfB)
   
SSW = 0
for j in range(k):
    SSWj = 0
    for i in range(int(sizes[j])):
        SSWj += np.power(samples[j][i] - means[j], 2)
    SSW += SSWj
dfW = int(np.sum(sizes) - k)
print('SSW           : %.4f' %SSW)
print('dfW           : %d' %dfW)

F = (SSB / dfB) / (SSW / dfW)
print('F statistic   : %.4f' %F)


critical_value = stats.f.ppf(q = 1 - alpha, dfn = dfB, dfd = dfW)
print('Critical value: %.4f' %critical_value)
conf_level     = stats.f.cdf(critical_value, dfn = dfB, dfd = dfW)

if (F < critical_value):
    print('(F <  critical value) ==> ACCEPT the H0 that the means are equal.')
else:
    print('(F >= critical value) ==> REJECT the H0 that the means are equal.')

##------------------------------------------------------------------------------
print('\n--------------------- Cách 2: Sử dụng hàm của Python ------------------')
##------------------------------------------------------------------------------    
fvalue, pvalue = stats.f_oneway(samples[0], samples[1], samples[2], samples[3])

print('p-value       : %.4f' %pvalue)
print('F statistic   : %.4f' %fvalue)

if (pvalue > 0.05):
    print('(pvalue >  0.05) ==> The means are equal.')
else:
    print('(F <= 0.05) ==> The means are NOT equal.')


----------- Cách 1: Tính toán "truyền thống" theo các công thức ---------
SSB           : 12486.4848
dfB           : 3
SSW           : 11464.5722
dfW           : 42
F statistic   : 15.2479
Critical value: 2.8270
(F >= critical value) ==> REJECT the H0 that the means are equal.

--------------------- Cách 2: Sử dụng hàm của Python ------------------
p-value       : 0.0000
F statistic   : 15.1391
(F <= 0.05) ==> The means are NOT equal.


In [36]:
#6.3. Kiểm định Tukey HSD
#Chuẩn bị dữ liệu theo statsmodels
df_melt = pd.melt(df.reset_index(), id_vars = ['index'], 
                  value_vars = ['A', 'B', 'C', 'D'])


df_melt.columns = ['index', 'samples', 'value']
df_melt.dropna(inplace=True)
model = ols('value ~ C(samples)', data = df_melt).fit()
m_comp = pairwise_tukeyhsd(endog = df_melt['value'], groups = df_melt['samples'], alpha = 0.05)
print(m_comp)

# ngoại trừ A&D, B&C, tất cả các so sánh cặp khác đều bác bỏ H0
# và chỉ ra sự khác biệt đáng kể về mặt thống kê.

 Multiple Comparison of Means - Tukey HSD, FWER=0.05  
group1 group2 meandiff p-adj   lower    upper   reject
------------------------------------------------------
     A      B -35.3667  0.001 -53.4103  -17.323   True
     A      C -36.9167  0.001 -54.0344 -19.7989   True
     A      D -11.7778 0.3415 -30.4132   6.8577  False
     B      C    -1.55    0.9 -20.4744  17.3744  False
     B      D  23.5889 0.0171   3.2814  43.8964   True
     C      D  25.1389 0.0068   5.6495  44.6283   True
------------------------------------------------------
